In [7]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import spacy
import string
import gensim
import operator
import ast

In [2]:
df_Product = pd.read_csv('D:\INPT2\Machine Learning\Search Engine\Data_Amazon_clean.csv')
df_Product = df_Product.drop(["ss","Unnamed1"], axis=1)

In [3]:
df_Product.head(3)

,name,price,images,rating,product_description,number_of_reviews,name_clean,category_clean
0,"Dirt Devil Simpli-Stik Vacuum Cleaner, 3-in-1 ...",$25.37,['https://m.media-amazon.com/images/I/71SXgQ8U...,3.9 out of 5,Home & Kitchen › Vacuums & Floor Care › Vacuum...,"25,316 global ratings","['vacuum', 'hand', 'bagless', 'vac', 'devil', ...","['vacuum', 'care', 'floor', 'broom', 'kitchen'..."
1,Tineco Floor ONE S3 Cordless Hardwood Floors C...,$399.99,['https://m.media-amazon.com/images/I/610REMZ7...,4.6 out of 5,Home & Kitchen › Vacuums & Floor Care › Vacuum...,"17,684 global ratings","['vacuum', 'multisurfac', 'cordless', 'floor',...","['vacuum', 'care', 'floor', 'broom', 'kitchen'..."
2,"Drop PLNT HNGR 4FT 3/8""12",NaN,['https://m.media-amazon.com/images/I/31GM6ows...,NaN,NaN,NaN,"['drop', 'hngr', 'plnt']",['nan']


In [13]:
print ('Cleaning and Tokenizing...')
#pd.concat([df_Product["name_clean"],df_Product["category_clean"]],axis=1)

df_Product["Prod_info"]=pd.DataFrame([i for i in df_Product["name_clean"]+df_Product["category_clean"]])
df_Product["Prod_info"]=df_Product["Prod_info"].apply(lambda x: list(set(ast.literal_eval(str(x).replace("][",',')))))
df_Product.head()

Cleaning and Tokenizing...


,name,price,images,rating,product_description,number_of_reviews,name_clean,category_clean,Prod_info
0,"Dirt Devil Simpli-Stik Vacuum Cleaner, 3-in-1 ...",$25.37,['https://m.media-amazon.com/images/I/71SXgQ8U...,3.9 out of 5,Home & Kitchen › Vacuums & Floor Care › Vacuum...,"25,316 global ratings","['vacuum', 'hand', 'bagless', 'vac', 'devil', ...","['vacuum', 'care', 'floor', 'broom', 'kitchen'...","[hand, bagless, simplistik, vacuum, home, elec..."
1,Tineco Floor ONE S3 Cordless Hardwood Floors C...,$399.99,['https://m.media-amazon.com/images/I/610REMZ7...,4.6 out of 5,Home & Kitchen › Vacuums & Floor Care › Vacuum...,"17,684 global ratings","['vacuum', 'multisurfac', 'cordless', 'floor',...","['vacuum', 'care', 'floor', 'broom', 'kitchen'...","[tineco, wet, smart, control, clean, broom, co..."
2,"Drop PLNT HNGR 4FT 3/8""12",NaN,['https://m.media-amazon.com/images/I/31GM6ows...,NaN,NaN,NaN,"['drop', 'hngr', 'plnt']",['nan'],"[drop, nan, hngr, plnt]"
3,"Cuisinart Waffle Maker, Double Belgian, Stainl...",$119.95,['https://m.media-amazon.com/images/I/81RjZKgQ...,4.8 out of 5,Home & Kitchen › Kitchen & Dining › Small Appl...,"13,004 global ratings","['steel', 'belgian', 'waffl', 'cuisinart', 'ma...","['applianc', 'waffl', 'kitchen', 'home', 'dine...","[small, steel, stainless, waffl, home, dine, i..."
4,Waterdrop DA29-00020B Refrigerator Water Filte...,$28.09,['https://m.media-amazon.com/images/I/71apo-WS...,4.7 out of 5,Appliances › Parts & Accessories › Refrigerato...,"3,070 global ratings","['pack', 'water', 'refriger', 'hafcinexp', 're...","['applianc', 'water', 'refriger', 'accessori',...","[replac, waterdrop, hafcinexp, refriger, water..."


In [14]:
product_plot = df_Product["Prod_info"]
product_plot[0:5]

0    [hand, bagless, simplistik, vacuum, home, elec...
1    [tineco, wet, smart, control, clean, broom, co...
2                              [drop, nan, hngr, plnt]
3    [small, steel, stainless, waffl, home, dine, i...
4    [replac, waterdrop, hafcinexp, refriger, water...
Name: Prod_info, dtype: object

In [15]:
from gensim import corpora

#creating term dictionary
dictionary = corpora.Dictionary(product_plot)

#filter out terms which occurs in less than 4 documents and more than 20% of the documents.
# NOTE: Since we have smaller dataset, we will keep this commented for now.

#dictionary.filter_extremes(no_below=4, no_above=0.2)

#list of few which which can be further removed
stoplist = set('hello and if this can would should could tell ask stop come go')
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist if stopword in dictionary.token2id]
dictionary.filter_tokens(stop_ids)

In [16]:
#print top 50 items from the dictionary with their unique token-id
dict_tokens = [[[dictionary[key], dictionary.token2id[dictionary[key]]] for key, value in dictionary.items() if key <= 50]]
print (dict_tokens)

[[['bagless', 0], ['broom', 1], ['care', 2], ['cleaner', 3], ['devil', 4], ['dirt', 5], ['electr', 6], ['floor', 7], ['hand', 8], ['home', 9], ['kitchen', 10], ['lightweight', 11], ['red', 12], ['simplistik', 13], ['small', 14], ['stick', 15], ['vac', 16], ['vacuum', 17], ['clean', 18], ['control', 19], ['cordless', 20], ['dri', 21], ['hardwood', 22], ['multisurfac', 23], ['one', 24], ['smart', 25], ['system', 26], ['tineco', 27], ['wet', 28], ['drop', 29], ['hngr', 30], ['nan', 31], ['plnt', 32], ['applianc', 33], ['belgian', 34], ['cuisinart', 35], ['dine', 36], ['doubl', 37], ['iron', 38], ['maker', 39], ['stainless', 40], ['steel', 41], ['waffl', 42], ['accessori', 43], ['filter', 44], ['hafcinexp', 45], ['pack', 46], ['part', 47], ['refriger', 48], ['replac', 49], ['samsung', 50]]]


In [18]:
corpus = [dictionary.doc2bow(desc) for desc in product_plot]

word_frequencies = [[(dictionary[id], frequency) for id, frequency in line] for line in corpus[0:3]]

print(word_frequencies)

[[('bagless', 1), ('broom', 1), ('care', 1), ('cleaner', 1), ('devil', 1), ('dirt', 1), ('electr', 1), ('floor', 1), ('hand', 1), ('home', 1), ('kitchen', 1), ('lightweight', 1), ('red', 1), ('simplistik', 1), ('small', 1), ('stick', 1), ('vac', 1), ('vacuum', 1)], [('broom', 1), ('care', 1), ('cleaner', 1), ('electr', 1), ('floor', 1), ('home', 1), ('kitchen', 1), ('lightweight', 1), ('stick', 1), ('vacuum', 1), ('clean', 1), ('control', 1), ('cordless', 1), ('dri', 1), ('hardwood', 1), ('multisurfac', 1), ('one', 1), ('smart', 1), ('system', 1), ('tineco', 1), ('wet', 1)], [('drop', 1), ('hngr', 1), ('nan', 1), ('plnt', 1)]]


In [19]:
movie_tfidf_model = gensim.models.TfidfModel(corpus, id2word=dictionary)
movie_lsi_model = gensim.models.LsiModel(movie_tfidf_model[corpus], id2word=dictionary, num_topics=300)

In [20]:
gensim.corpora.MmCorpus.serialize('movie_tfidf_model_mm', movie_tfidf_model[corpus])
gensim.corpora.MmCorpus.serialize('movie_lsi_model_mm',movie_lsi_model[movie_tfidf_model[corpus]])

In [21]:
#Load the indexed corpus
movie_tfidf_corpus = gensim.corpora.MmCorpus('movie_tfidf_model_mm')
movie_lsi_corpus = gensim.corpora.MmCorpus('movie_lsi_model_mm')

print(movie_tfidf_corpus)
print(movie_lsi_corpus)

MmCorpus(292 documents, 1552 features, 5183 non-zero entries)
MmCorpus(292 documents, 268 features, 77966 non-zero entries)


In [22]:
from gensim.similarities import MatrixSimilarity

%time movie_index = MatrixSimilarity(movie_lsi_corpus, num_features = movie_lsi_corpus.num_terms)

CPU times: total: 141 ms
Wall time: 198 ms


In [23]:
from operator import itemgetter

In [30]:
spacy_nlp = spacy.load('en_core_web_sm')

#create list of punctuations and stopwords
punctuations = string.punctuation
stop_words = spacy.lang.en.stop_words.STOP_WORDS

#function for data cleaning and processing
#This can be further enhanced by adding / removing reg-exps as desired.

def spacy_tokenizer(sentence):
 
    #remove distracting single quotes
    sentence = re.sub('\'','',sentence)

    #remove digits adnd words containing digits
    sentence = re.sub('\w*\d\w*','',sentence)

    #replace extra spaces with single space
    sentence = re.sub(' +',' ',sentence)

    #remove unwanted lines starting from special charcters
    sentence = re.sub(r'\n: \'\'.*','',sentence)
    sentence = re.sub(r'\n!.*','',sentence)
    sentence = re.sub(r'^:\'\'.*','',sentence)
    
    #remove non-breaking new line characters
    sentence = re.sub(r'\n',' ',sentence)
    
    #remove punctunations
    sentence = re.sub(r'[^\w\s]',' ',sentence)
    
    #creating token object
    tokens = spacy_nlp(sentence)
    
    #lower, strip and lemmatize
    tokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in tokens]
    
    #remove stopwords, and exclude words less than 2 characters
    tokens = [word for word in tokens if word not in stop_words and word not in punctuations and len(word) > 2]
    
    #return tokens
    return tokens

In [40]:

def search_similar_products(search_term):

    query_bow = dictionary.doc2bow(spacy_tokenizer(search_term))
    query_tfidf = movie_tfidf_model[query_bow]
    query_lsi = movie_lsi_model[query_tfidf]

    movie_index.num_best = 5

    movies_list = movie_index[query_lsi]

    movies_list.sort(key=itemgetter(1), reverse=True)
    movie_names = []

    for j, movie in enumerate(movies_list):

        movie_names.append (
            {
                'Relevance': round((movie[1] * 100),2),
                'Movie Title': df_Product['name'][movie[0]],
                'images': df_Product['images'][movie[0]]
            }

        )
        if j == (movie_index.num_best-1):
            break

    return pd.DataFrame(movie_names, columns=['Relevance','Movie Title','images'])

In [42]:
pd.set_option('display.max_colwidth', None)

In [50]:
# search for products tiles that are related to below search parameters
search_similar_products('phone samsung')

,Relevance,Movie Title,images
0,51.33,Samsung Dryer Belt 6602-001655,"['https://m.media-amazon.com/images/I/61Pw4Nzu32L._AC_SX355_.jpg', 'https://m.media-amazon.com/images/I/61Pw4Nzu32L._AC_SX569_.jpg', 'https://m.media-amazon.com/images/I/61Pw4Nzu32L._AC_SX425_.jpg', 'https://m.media-amazon.com/images/I/61Pw4Nzu32L._AC_SX466_.jpg', 'https://m.media-amazon.com/images/I/61Pw4Nzu32L._AC_SX679_.jpg', 'https://m.media-amazon.com/images/I/61Pw4Nzu32L._AC_SX450_.jpg', 'https://m.media-amazon.com/images/I/61Pw4Nzu32L._AC_SX522_.jpg']"
1,47.01,"USB C Car Charger, 2Pack 30W Type C Fast Charging Cigarette Lighter Adapter for iPhone 13 Pro Max/12 Pro Max/Mini/SE/11 Pro Max/XR/XS/X/8,Samsung Galaxy S22 S21 S20 A12 A32 A42,Pixel 6 Pro 5 4 3A 3XL","['https://m.media-amazon.com/images/I/61-0rY-+USS._AC_SX425_.jpg', 'https://m.media-amazon.com/images/I/61-0rY-+USS._AC_SX522_.jpg', 'https://m.media-amazon.com/images/I/61-0rY-+USS._AC_SX569_.jpg', 'https://m.media-amazon.com/images/I/61-0rY-+USS._AC_SX385_.jpg', 'https://m.media-amazon.com/images/I/61-0rY-+USS._AC_SX679_.jpg', 'https://m.media-amazon.com/images/I/61-0rY-+USS._AC_SX342_.jpg', 'https://m.media-amazon.com/images/I/61-0rY-+USS._AC_SX466_.jpg']"
2,41.04,"Temdan for Samsung Galaxy S22 Ultra Case Waterproof, Built in Screen Protector 360° Full Body Heavy Duty Shockproof IP68 Waterproof Galaxy S22 Ultra Case for Samsung S22 Ultra 5G 6.8 inch","['https://m.media-amazon.com/images/I/81e16yiwYPL._AC_SX425_.jpg', 'https://m.media-amazon.com/images/I/81e16yiwYPL._AC_SX522_.jpg', 'https://m.media-amazon.com/images/I/81e16yiwYPL._AC_SX569_.jpg', 'https://m.media-amazon.com/images/I/81e16yiwYPL._AC_SX466_.jpg', 'https://m.media-amazon.com/images/I/81e16yiwYPL._AC_SX342_.jpg', 'https://m.media-amazon.com/images/I/81e16yiwYPL._AC_SX679_.jpg', 'https://m.media-amazon.com/images/I/81e16yiwYPL._AC_SX385_.jpg']"
3,41.04,"Temdan for Samsung Galaxy S22 Ultra Case Waterproof, Built in Screen Protector 360° Full Body Heavy Duty Shockproof IP68 Waterproof Galaxy S22 Ultra Case for Samsung S22 Ultra 5G 6.8 inch","['https://m.media-amazon.com/images/I/716avFNvAcL._AC_SX425_.jpg', 'https://m.media-amazon.com/images/I/716avFNvAcL._AC_SX385_.jpg', 'https://m.media-amazon.com/images/I/716avFNvAcL._AC_SX569_.jpg', 'https://m.media-amazon.com/images/I/716avFNvAcL._AC_SX342_.jpg', 'https://m.media-amazon.com/images/I/716avFNvAcL._AC_SX466_.jpg', 'https://m.media-amazon.com/images/I/716avFNvAcL._AC_SX679_.jpg', 'https://m.media-amazon.com/images/I/716avFNvAcL._AC_SX522_.jpg']"
4,41.04,"Temdan for Samsung Galaxy S22 Ultra Case Waterproof, Built in Screen Protector 360° Full Body Heavy Duty Shockproof IP68 Waterproof Galaxy S22 Ultra Case for Samsung S22 Ultra 5G 6.8 inch","['https://m.media-amazon.com/images/I/71BxFojdSIL._AC_SX385_.jpg', 'https://m.media-amazon.com/images/I/71BxFojdSIL._AC_SX466_.jpg', 'https://m.media-amazon.com/images/I/71BxFojdSIL._AC_SX342_.jpg', 'https://m.media-amazon.com/images/I/71BxFojdSIL._AC_SX679_.jpg', 'https://m.media-amazon.com/images/I/71BxFojdSIL._AC_SX522_.jpg', 'https://m.media-amazon.com/images/I/71BxFojdSIL._AC_SX425_.jpg', 'https://m.media-amazon.com/images/I/71BxFojdSIL._AC_SX569_.jpg']"
